In [1]:
from grtoolkit.Math import *
import re

In [20]:
expr = 'eq.append("Eq(w,integrate(C*v,(v,v0,v1))-integrate(C*v,(v,v0,v1)))")'
# print(preSympifySub(expr,v="99*red_ballons"))

expr.find("integrate")


16

In [52]:
# expr.find("(")

integrations = [m.start() for m in re.finditer('integrate', expr)]
open_brackets = [m.start() for m in re.finditer('\(', expr)]
close_brackets = [m.start() for m in re.finditer('\)', expr)]
commas = [m.start() for m in re.finditer(',', expr)]
# close_brackets = close_brackets[::-1]

In [53]:
print(integrations)
print(open_brackets)
print(close_brackets)
print(commas)

all_open_brackets = [[open,'o'] for open in open_brackets]
all_close_brackets = [[close,'c'] for close in close_brackets]
all_brackets = sorted(all_open_brackets + all_close_brackets)

print(all_brackets)

[16, 41]
[9, 13, 25, 30, 50, 55]
[38, 39, 63, 64, 65, 67]
[15, 29, 32, 35, 54, 57, 60]
[[9, 'o'], [13, 'o'], [25, 'o'], [30, 'o'], [38, 'c'], [39, 'c'], [50, 'o'], [55, 'o'], [63, 'c'], [64, 'c'], [65, 'c'], [67, 'c']]


In [37]:
copyOpen = open_brackets
taken = list()
bracket_pairs = list()


for b in range(len(all_brackets)):
    if all_brackets[b][1] == "c":
        i = 1
        while True:
            if all_brackets[b-i][1] == "o" and b-i not in taken:
                bracket_pairs.append([all_brackets[b-i][0],all_brackets[b][0]])
                taken.append(b-i)
                break
            i+=1
sorted_bracket_pairs = sorted(bracket_pairs)
print(sorted(sorted_bracket_pairs))

[[9, 67], [13, 65], [25, 39], [30, 38], [50, 64], [55, 63]]


In [50]:
master_func_bracket_pairs = list()

# for i in range(len(bracket_pairs)):
#     # print(bracket_pairs[i])
#     for func in integrations:
#         if func > bracket_pairs[i][0] and func < bracket_pairs[i+1][0]:
#             ignore_bracket_pairs.append(bracket_pairs[i+2])

for i in range(len(sorted_bracket_pairs)):
    # print(bracket_pairs[i])
    for func in integrations:
        if func > sorted_bracket_pairs[i][0] and func < sorted_bracket_pairs[i+1][0]:
            master_func_bracket_pairs.append(sorted_bracket_pairs[i+1])

print(master_func_bracket_pairs)

[[25, 39], [50, 64]]


In [39]:
expr

'eq.append("Eq(w,integrate(C*v,(v,v0,v1))-integrate(C*v,(v,v0,v1)))")'

In [51]:
print(expr[master_func_bracket_pairs[0][0]:master_func_bracket_pairs[0][1]+1])
print(expr[master_func_bracket_pairs[1][0]:master_func_bracket_pairs[1][1]+1])

(C*v,(v,v0,v1))
(C*v,(v,v0,v1))


In [ ]:
# Needs to be comma sensitive